## Imports

In [1]:
import pandas as pd
import numpy as np
import geopandas as gp
import os
import maup
from op_verification.reference_data import *
import boto3
from s3_paths import *
import io
pd.set_option("display.max_columns", None)
import warnings
warnings.filterwarnings('ignore', 'GeoSeries.isna', UserWarning)


maup.progress.enabled = True

## Functions

In [70]:
#Function to adjust Census VAP to account for incarcerated pop by subtracting and then assign 0 to any negative values
def modify_P0040001_P0050003(block_gdf):
    block_gdf['P0040001-P0050003'] = block_gdf['P0040001'] - block_gdf['P0050003']
    print("Error raised iff the number of incarcerated blocks with non-zero population does NOT equal the number of modified vap blocks.")
    assert (len(block_gdf[block_gdf['P0050003']!=0]) - len(block_gdf[block_gdf['P0040001']!=block_gdf['P0040001-P0050003']])) == 0
    block_gdf.loc[block_gdf['P0040001-P0050003']<0, 'P0040001-P0050003'] = 0

    return block_gdf


#Functions for running maup
def check_valid_rows(block_gdf, precinct_gdf):
    prec_geom = precinct_gdf.geometry
    valid_rows = precinct_gdf[~(prec_geom.isna() | prec_geom.is_empty)]
    print('valid precinct rows: ', valid_rows.shape)

    block_geom = block_gdf.geometry
    block_valid_rows = block_gdf[~(block_geom.isna() | block_geom.is_empty)]
    print('valid block rows: ', block_valid_rows.shape)
    

def fix_buffer(gdf):
    """
    return (GeoDataFrame) with the 'bufer(0) trick' applied
    :gdf: (GeoDataFrame) object
    Can be useful when trying to mitigate 'self-intersection' issues
    """
    buffered = gdf.buffer(0)
    gdf.drop(columns=["geometry"])
    # gdf['geometry'] = gdf.apply(lambda x: x.geometry.buffer(0), axis=1)
    gdf["geometry"] = buffered
    return gdf


def maup_assignment_series(block_gdf, precinct_gdf):
    block_gdf = block_gdf.to_crs(precinct_gdf.crs)
    print('block_gdf shape: ', block_gdf.shape, '\n precinct_gdf shape: ', precinct_gdf.shape)

    block_gdf["maup_assignment"] = maup.assign(
        fix_buffer(block_gdf), fix_buffer(precinct_gdf)
    )
    print(block_gdf[block_gdf['maup_assignment'].isna()].shape[0], " null assignments in maup series")
    return block_gdf


def maup_pre_vote_setup(block_gdf, precinct_gdf, PRECID_block):
    block_gdf = block_gdf.to_crs(precinct_gdf.crs)
    bgdf = maup_assignment_series(block_gdf, precinct_gdf)
    assert bgdf['maup_assignment'].isna().any() == False
    bgdf[PRECID_block] = bgdf['maup_assignment'].map(lambda idx: str(precinct_gdf.loc[idx]["UNIQUE_ID"]))
    
    return bgdf

    
def maup_assign_labels(block_gdf, precinct_gdf, precinctid_input, distid_input):
    #Assign precinct IDs to block file using assign series
    block_gdf[precinctid_input] = block_gdf["maup_assignment"].map(
        lambda idx: str(precinct_gdf.loc[idx]["UNIQUE_ID"])
    )
    if "CONG_DIST" in precinct_gdf.columns:
        block_gdf[distid_input] = block_gdf["maup_assignment"].map(
            lambda idx: str(precinct_gdf.loc[idx]["CONG_DIST"])
        )
    if "SLDL_DIST" in precinct_gdf.columns:
        block_gdf[distid_input] = block_gdf["maup_assignment"].map(
            lambda idx: str(precinct_gdf.loc[idx]["SLDL_DIST"])
        )
    if "SLDU_DIST" in precinct_gdf.columns:
        block_gdf[distid_input] = block_gdf["maup_assignment"].map(
            lambda idx: str(precinct_gdf.loc[idx]["SLDU_DIST"])
        )
    print("Label assignment complete")

    return block_gdf


def election_cols(gdf):
    election_cols = list(gdf.columns[gdf.columns.str.startswith('G2')|gdf.columns.str.startswith('GC')|
                                     gdf.columns.str.startswith('GR')|gdf.columns.str.startswith('GS')])
    return election_cols 


def assign_votes(variables, election_columns, precinct_gdf, block_gdf):
    precinct_gdf[variables] = block_gdf[variables].groupby(block_gdf["maup_assignment"]).sum()
    print(variables, ' added to precinct_gdf based on maup assignment')
    bl_to_prec_weights = block_gdf[variables] / block_gdf["maup_assignment"].map(precinct_gdf[variables])
    block_votes = block_gdf.copy(deep=True)
    block_votes[election_columns] = maup.prorate(
        block_gdf["maup_assignment"], precinct_gdf[election_columns], bl_to_prec_weights
    )
    return block_votes


def run_maup(block_gdf, precinct_gdf, precinctid_input, pop_variable):
    #Assign identifiers
    block_gdf = maup_assignment_series(block_gdf, precinct_gdf)
    block_gdf = maup_assign_labels(block_gdf, precinct_gdf, precinctid_input)

    #Assign votes
    election_columns = election_cols(precinct_gdf)
    block_votes = assign_votes(pop_variable, election_columns, precinct_gdf, block_gdf)
    
    return block_votes   


def column_total_check(election_columns, block_gdf, precinct_gdf):
    for val in election_columns:
        vote_dif = block_gdf[val].sum()-precinct_gdf[val].sum()
        if (abs(vote_dif) <=1e-1):
            #print(val+": EQUAL", ' - total: ', 'block:', str(block_gdf[val].sum()), 'prec:', str(precinct_gdf[val].sum()), 'diff:', block_gdf[val].sum()-precinct_gdf[val].sum())
            continue
        else:
            print(val+": DIFFERENCE OF " + str(vote_dif)+ " VOTES", ' - block total: ', str(block_gdf[val].sum()), ', precinct total: ', str(precinct_gdf[val].sum()))  
            
            
def precinct_sum_check(prec_gdf, block_gdf, blk_prec_id):
    prec_gpbyprec = prec_gdf.groupby(['UNIQUE_ID']).sum()
    blk_gpbyprec = block_gdf.groupby([blk_prec_id]).sum()
    prec_w_difs_list = []
    
    prec_gpbyprec.sort_values(by='UNIQUE_ID',inplace=True)
    prec_gpbyprec.reset_index(inplace = True)
    blk_gpbyprec.sort_values(by= blk_prec_id, inplace=True)
    blk_gpbyprec.reset_index(inplace = True)
    
    for i in range(0, prec_gdf['UNIQUE_ID'].nunique()):
        for col in election_cols(prec_gdf):
            prec_diff = round(blk_gpbyprec[col].iloc[i], 1) - round(prec_gpbyprec[col].iloc[i], 1)
            if (prec_diff!=0):
                prec_w_difs_list.append(prec_gpbyprec.index[i])
                print("prec id: ", blk_gpbyprec.index[i], prec_gpbyprec.index[i], col, "block: ", round(blk_gpbyprec[col].iloc[i],1), 
                      "prec: ", round(prec_gpbyprec[col].iloc[i],1), "diff: ", prec_diff)
    print("precs to check: ", set(prec_w_difs_list))
    print("Precinct check complete!")
            

## Base file set up

In [3]:
#Block file set up
s3_client = boto3.client('s3')
census_block_gdf = gp.read_file(f'zip+{census_block}')[['GEOID20', 'COUNTYFP20', 'NAME20','P0010001', 'P0020001', 'P0040001', 'P0050003', 'geometry']]
census_block_gdf = modify_P0040001_P0050003(census_block_gdf)


#Precinct file set up
vtd = gp.read_file('./tx_2022_gen_tx_vtd')
print(vtd.shape)

Error raised iff the number of incarcerated blocks with non-zero population does NOT equal the number of modified vap blocks.
(9634, 371)


## Maup assign

In [4]:
#vtd.crs = "EPSG:4269"
bvtd = maup_pre_vote_setup(census_block_gdf, vtd, "PREC")

block_gdf shape:  (668757, 9) 
 precinct_gdf shape:  (9634, 371)


100%|██████████| 9634/9634 [03:34<00:00, 44.98it/s]  


0  null assignments in maup series


In [5]:
bvtd_votes = assign_votes("P0040001-P0050003", election_cols(vtd), vtd, bvtd)
print("vtd disag complete")

P0040001-P0050003  added to precinct_gdf based on maup assignment
vtd disag complete


In [6]:
bvtd_votes.isnull().any()

GEOID20       False
COUNTYFP20    False
NAME20        False
P0010001      False
P0020001      False
              ...  
GSU26RMUR      True
GSU27DLAM      True
GSU27RHIN      True
GSU29DBLA      True
GSU29RZUB      True
Length: 377, dtype: bool

In [7]:
vtd[vtd['P0040001-P0050003']==0].sum()[vtd[vtd['P0040001-P0050003']==0].sum()!=0]

UNIQUE_ID     027-0117029-1090029-1106029-1126029-1131029-11...
COUNTYFP      0270290290290290290290290290290290290290290290...
County        BellBexarBexarBexarBexarBexarBexarBexarBexarBe...
TX_VTD        0117109011061126113111372109211621292132215521...
G22AGRDHAY                                                  174
                                    ...                        
GSU18RKOL                                                    15
GSU22LSCH                                                    40
GSU22RBIR                                                    37
GSU25DWAL                                                    76
GSU25RCAM                                                    45
Length: 76, dtype: object

## Checks

In [8]:
#Column total checks
column_total_check(election_cols(bvtd_votes), bvtd_votes, vtd)

G22AGRDHAY: DIFFERENCE OF -174.00000000093132 VOTES  - block total:  3474523.999999999 , precinct total:  3474698
G22AGRRMIL: DIFFERENCE OF -123.0 VOTES  - block total:  4480163.0 , precinct total:  4480286
G22ATGDGAR: DIFFERENCE OF -175.00000000046566 VOTES  - block total:  3498100.9999999995 , precinct total:  3498276
G22ATGLASH: DIFFERENCE OF -11.000000000058208 VOTES  - block total:  233756.99999999994 , precinct total:  233768
G22ATGRPAX: DIFFERENCE OF -115.0 VOTES  - block total:  4278744.0 , precinct total:  4278859
G22COMDDUD: DIFFERENCE OF -167.0 VOTES  - block total:  3265779.0 , precinct total:  3265946
G22COMLECH: DIFFERENCE OF -13.000000000058208 VOTES  - block total:  212232.99999999994 , precinct total:  212246
G22COMRHEG: DIFFERENCE OF -119.0 VOTES  - block total:  4496264.0 , precinct total:  4496383
G22GOVDORO: DIFFERENCE OF -185.99999999906868 VOTES  - block total:  3554586.000000001 , precinct total:  3554772
G22GOVLTIP: DIFFERENCE OF -6.000000000029104 VOTES  - blo

In [9]:
#Precinct total checks
precinct_sum_check(vtd, bvtd_votes, "PREC")

prec id:  622 622 G22AGRDHAY block:  0.0 prec:  73 diff:  -73.0
prec id:  622 622 G22AGRRMIL block:  0.0 prec:  43 diff:  -43.0
prec id:  622 622 G22ATGDGAR block:  0.0 prec:  76 diff:  -76.0
prec id:  622 622 G22ATGLASH block:  0.0 prec:  1 diff:  -1.0
prec id:  622 622 G22ATGRPAX block:  0.0 prec:  41 diff:  -41.0
prec id:  622 622 G22COMDDUD block:  0.0 prec:  70 diff:  -70.0
prec id:  622 622 G22COMLECH block:  0.0 prec:  2 diff:  -2.0
prec id:  622 622 G22COMRHEG block:  0.0 prec:  45 diff:  -45.0
prec id:  622 622 G22GOVDORO block:  0.0 prec:  80 diff:  -80.0
prec id:  622 622 G22GOVRABB block:  0.0 prec:  38 diff:  -38.0
prec id:  622 622 G22LANDKLE block:  0.0 prec:  75 diff:  -75.0
prec id:  622 622 G22LANRBUC block:  0.0 prec:  42 diff:  -42.0
prec id:  622 622 G22LTGDCOL block:  0.0 prec:  76 diff:  -76.0
prec id:  622 622 G22LTGLSTE block:  0.0 prec:  2 diff:  -2.0
prec id:  622 622 G22LTGRPAT block:  0.0 prec:  39 diff:  -39.0
prec id:  622 622 GCCA05DHUF block:  0.0 prec:

In [37]:
index_zero_pop_prec = {3257, 1241, 1830, 1288, 3789, 622, 5008, 2577, 7986, 7987, 8528, 6805, 7957, 2555, 2552, 761, 8021, 1275}

In [11]:
vtd['P0040001-P0050003'][vtd.index.isin(index_zero_pop_prec)]

622     0
761     0
1241    0
1275    0
1288    0
1830    0
2552    0
2555    0
2577    0
3257    0
3789    0
5008    0
6805    0
7957    0
7986    0
7987    0
8021    0
8528    0
Name: P0040001-P0050003, dtype: int64

In [13]:
vtd['UNIQUE_ID'][vtd.index.isin(index_zero_pop_prec)]

622      029-3019
761      029-3156
1241     041-0051
1275     041-0085
1288     041-0097
1830     085-0202
2552     113-3111
2555     113-3114
2577     113-32T0
3257     139-1004
3789     157-4170
5008     201-0791
6805    327-0004B
7957     439-1376
7986     439-1455
7987     439-1457
8021     439-1603
8528     439-4458
Name: UNIQUE_ID, dtype: object

In [17]:
zero_pop_precs = bvtd_votes[bvtd_votes['PREC'].isin(list(vtd['UNIQUE_ID'][vtd.index.isin(index_zero_pop_prec)]))]

In [19]:
os.mkdir("./zero_pop_precs/")
zero_pop_precs.to_file("./zero_pop_precs/zero_pop_precs.shp")

<ipython-input-19-3839795a172a>:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  zero_pop_precs.to_file("./zero_pop_precs/zero_pop_precs.shp")


In [28]:
vtd[vtd.index.isin(index_zero_pop_prec)].to_file("./zero_pop_precs/prec_zero_pop_precs.shp")

<ipython-input-28-fffd1b0f1c4b>:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  vtd[vtd.index.isin({3257, 1241, 1830, 1288, 3789, 622, 5008, 2577, 7986, 7987, 8528, 6805, 7957, 2555, 2552, 761, 8021, 1275})].to_file("./zero_pop_precs/prec_zero_pop_precs.shp")


In [29]:
prec_zero_pop = vtd[vtd.index.isin(index_zero_pop_prec)]

In [31]:
prec_zero_pop.sum()[prec_zero_pop.sum()!=0]

UNIQUE_ID     029-3019029-3156041-0051041-0085041-0097085-02...
COUNTYFP      0290290410410410851131131131391572013274394394...
County        BexarBexarBrazosBrazosBrazosCollinDallasDallas...
TX_VTD        3019315600510085009702023111311432T01004417007...
G22AGRDHAY                                                  174
                                    ...                        
GSU18RKOL                                                    15
GSU22LSCH                                                    40
GSU22RBIR                                                    37
GSU25DWAL                                                    76
GSU25RCAM                                                    45
Length: 76, dtype: object

In [26]:
zero_pop_precs[['PREC','P0040001-P0050003', 'P0040001', 'P0050003']].groupby('PREC').sum()

,P0040001-P0050003,P0040001,P0050003
PREC,,,
029-3019,0,0,0
029-3156,0,0,0
041-0051,0,0,0
041-0085,0,0,0
041-0097,0,0,0
085-0202,0,0,0
113-3111,0,0,0
113-3114,0,0,0
113-32T0,0,0,0


In [35]:
prec_zero_pop.columns

Index(['UNIQUE_ID', 'COUNTYFP', 'County', 'TX_VTD', 'G22AGRDHAY', 'G22AGRRMIL',
       'G22ATGDGAR', 'G22ATGLASH', 'G22ATGRPAX', 'G22COMDDUD',
       ...
       'GSU25DWAL', 'GSU25RCAM', 'GSU26DMEN', 'GSU26RMUR', 'GSU27DLAM',
       'GSU27RHIN', 'GSU29DBLA', 'GSU29RZUB', 'geometry', 'P0040001-P0050003'],
      dtype='object', length=372)

## Reassign 0 pop vtds

In [38]:
#Subset blocks that need to be updated
bvtd_for_reassign = bvtd[bvtd['PREC'].isin(list(vtd['UNIQUE_ID'][vtd.index.isin(index_zero_pop_prec)]))]
bvtd_for_reassign['P0040001-P0050003_mod'] = 1

bvtd_reassign_votes = assign_votes("P0040001-P0050003_mod", election_cols(prec_zero_pop), prec_zero_pop, bvtd_for_reassign)
print("vtd disag complete")

/Users/lilyfalk/anaconda3/envs/pdv_env/lib/python3.9/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


P0040001-P0050003_mod  added to precinct_gdf based on maup assignment
vtd disag complete


In [39]:
#Column total checks
column_total_check(election_cols(bvtd_reassign_votes), bvtd_reassign_votes, prec_zero_pop)

In [40]:
#Precinct total checks
precinct_sum_check(prec_zero_pop, bvtd_reassign_votes, "PREC")

precs to check:  set()
Precinct check complete!


In [44]:
set(bvtd_reassign_votes.columns)-set(bvtd_votes)

{'P0040001-P0050003_mod'}

In [46]:
list(bvtd_reassign_votes.columns)

['GEOID20',
 'COUNTYFP20',
 'NAME20',
 'P0010001',
 'P0020001',
 'P0040001',
 'P0050003',
 'geometry',
 'P0040001-P0050003',
 'maup_assignment',
 'PREC',
 'P0040001-P0050003_mod',
 'G22AGRDHAY',
 'G22AGRRMIL',
 'G22ATGDGAR',
 'G22ATGLASH',
 'G22ATGRPAX',
 'G22COMDDUD',
 'G22COMLECH',
 'G22COMRHEG',
 'G22GOVDORO',
 'G22GOVGBAR',
 'G22GOVLTIP',
 'G22GOVOWRI',
 'G22GOVRABB',
 'G22LANDKLE',
 'G22LANGMOL',
 'G22LANOMEN',
 'G22LANRBUC',
 'G22LTGDCOL',
 'G22LTGLSTE',
 'G22LTGRPAT',
 'GCCA05DHUF',
 'GCCA05RWAL',
 'GCCA06DJOH',
 'GCCA06RMCC',
 'GCON01DJEF',
 'GCON01RMOR',
 'GCON02DFUL',
 'GCON02RCRE',
 'GCON03DSRI',
 'GCON03LCLA',
 'GCON03RSEL',
 'GCON04DOME',
 'GCON04LSIM',
 'GCON04RFAL',
 'GCON05DHIL',
 'GCON05LHAL',
 'GCON05OTOR',
 'GCON05RGOO',
 'GCON07DFLE',
 'GCON07RTEA',
 'GCON08DJON',
 'GCON08LERI',
 'GCON08RLUT',
 'GCON09DGRE',
 'GCON09RLEO',
 'GCON10DNUN',
 'GCON10LKEL',
 'GCON10RMCC',
 'GCON12DHUN',
 'GCON12RGRA',
 'GCON13DBRO',
 'GCON13RJAC',
 'GCON14DWIL',
 'GCON14RWEB',
 'GCON15DV

In [47]:
bvtd_no_reassign_votes = bvtd_votes[~bvtd_votes['PREC'].isin(list(vtd['UNIQUE_ID'][vtd.index.isin(index_zero_pop_prec)]))]
#concatenate the first run of block df with second
concat = pd.concat([bvtd_no_reassign_votes, bvtd_reassign_votes], keys = list(bvtd_reassign_votes.columns))
print("before shape: ",bvtd_votes.shape, " after shape: ",concat.shape)

before shape:  (668757, 377)  after shape:  (668757, 378)


In [48]:
#Column total checks
column_total_check(election_cols(concat), concat, vtd)

In [49]:
#Precinct total checks
precinct_sum_check(vtd, concat, "PREC")

precs to check:  set()
Precinct check complete!


In [58]:
concat.drop(concat.index)

,,GEOID20,COUNTYFP20,NAME20,P0010001,P0020001,P0040001,P0050003,geometry,P0040001-P0050003,maup_assignment,PREC,G22AGRDHAY,G22AGRRMIL,G22ATGDGAR,G22ATGLASH,G22ATGRPAX,G22COMDDUD,G22COMLECH,G22COMRHEG,G22GOVDORO,G22GOVGBAR,G22GOVLTIP,G22GOVOWRI,G22GOVRABB,G22LANDKLE,G22LANGMOL,G22LANOMEN,G22LANRBUC,G22LTGDCOL,G22LTGLSTE,G22LTGRPAT,GCCA05DHUF,GCCA05RWAL,GCCA06DJOH,GCCA06RMCC,GCON01DJEF,GCON01RMOR,GCON02DFUL,GCON02RCRE,GCON03DSRI,GCON03LCLA,GCON03RSEL,GCON04DOME,GCON04LSIM,GCON04RFAL,GCON05DHIL,GCON05LHAL,GCON05OTOR,GCON05RGOO,GCON07DFLE,GCON07RTEA,GCON08DJON,GCON08LERI,GCON08RLUT,GCON09DGRE,GCON09RLEO,GCON10DNUN,GCON10LKEL,GCON10RMCC,GCON12DHUN,GCON12RGRA,GCON13DBRO,GCON13RJAC,GCON14DWIL,GCON14RWEB,GCON15DVAL,GCON15LLEO,GCON15RDEL,GCON16DESC,GCON16RARM,GCON17DWOO,GCON17RSES,GCON18DJAC,GCON18IDUN,GCON18LKUR,GCON18RMON,GCON19ILEW,GCON19RARR,GCON20DCAS,GCON20OJON,GCON20RSIN,GCON21DZAP,GCON21RROY,GCON22DJOR,GCON22LLEB,GCON22OSQU,GCON22RNEH,GCON23DLIR,GCON23ILOP,GCON23RGON,GCON24DMCD,GCON24RVAN,GCON26LKOL,GCON26RBUR,GCON27DPER,GCON27RCLO,GCON28DCUE,GCON28RGAR,GCON29DGAR,GCON29RSCH,GCON30DCRO,GCON30IMAN,GCON30LGRA,GCON30OWAL,GCON30RROD,GCON32DALL,GCON32RSWA,GCON33DVEA,GCON33LASH,GCON33RGIL,GCON34DGON,GCON34IROY,GCON34RFLO,GCON35DCAS,GCON35RMCQ,GCON36DHAI,GCON36RBAB,GCON37DDOG,GCON37LPAT,GCON37OTAY,GCON37RGAR,GCON38DKLU,GCON38IDEJ,GCON38RHUN,GRRC01DWAR,GRRC01GCRO,GRRC01LDIE,GRRC01RCHR,GSL004LSAV,GSL004RBEL,GSL006DGRA,GSL006RSCH,GSL008LADA,GSL008RHAR,GSL009DROG,GSL009RASH,GSL013DDAV,GSL013RORR,GSL014LMIL,GSL014RRAN,GSL015DJOH,GSL015RTOT,GSL017DEDE,GSL017ICUR,GSL017RGER,GSL019DBAG,GSL019RTRO,GSL020DCAM,GSL020RWIL,GSL022DHAY,GSL022RRAN,GSL023DHEN,GSL023RLEO,GSL024DCRE,GSL024LMCC,GSL024RBON,GSL026DLEE,GSL026RJET,GSL027DREY,GSL027RGIL,GSL028DADR,GSL028RGAT,GSL031DGUT,GSL031RGUI,GSL033DLYN,GSL033RHOL,GSL034DHER,GSL034RVAU,GSL035DLON,GSL035RROS,GSL037DVIL,GSL037RLOP,GSL039DMAR,GSL039RGAR,GSL041DGUE,GSL041RGUE,GSL042DRAY,GSL042RBRE,GSL044DBOH,GSL044RKUE,GSL045DZWI,GSL045RLOP,GSL046DCOL,GSL046LKOS,GSL046RSTR,GSL047DGOO,GSL047RMCC,GSL048DHOW,GSL048LMCC,GSL049DHIN,GSL049LROB,GSL049RGRI,GSL050DTAL,GSL050LBRO,GSL050RJOH,GSL051DFLO,GSL051RREY,GSL052DECH,GSL052RHAR,GSL053DHER,GSL053RMUR,GSL054DHIL,GSL054RBUC,GSL055DSAN,GSL055RSHI,GSL056DSHA,GSL056RAND,GSL057LHAM,GSL057RHAY,GSL061DKIN,GSL061RFRA,GSL063DWOO,GSL063RBUM,GSL065DVER,GSL065RTHI,GSL066DRIN,GSL066RSHA,GSL067DMOR,GSL067RLEA,GSL069DCOP,GSL069LNEU,GSL069RFRA,GSL070DPLE,GSL070RJOL,GSL071DGOO,GSL071RLAM,GSL073DCAL,GSL073RISA,GSL074DMOR,GSL074RPAR,GSL075DGON,GSL075LMUL,GSL076DLAL,GSL076RMAT,GSL085DBAG,GSL085LMIL,GSL085RKIT,GSL087LHEA,GSL087RPRI,GSL092DBHO,GSL092RLIV,GSL093DCHO,GSL093RSCH,GSL094DSHE,GSL094RTIN,GSL095DCOL,GSL095RMON,GSL097DMCL,GSL097RGOL,GSL098DELK,GSL098RCAP,GSL099DCOF,GSL099RGER,GSL100DJON,GSL100LROB,GSL102DRAM,GSL102RFIS,GSL103DANC,GSL103IARR,GSL105DMEZ,GSL105RMEA,GSL107DNEA,GSL107LNEW,GSL108DGIN,GSL108RMEY,GSL111DDAV,GSL111RFLO,GSL112DCUR,GSL112RBUT,GSL114DBRY,GSL114RLAM,GSL115DJOH,GSL115RDEN,GSL117DCOR,GSL117RSCH,GSL118DRAM,GSL118RLUJ,GSL119DCAM,GSL119LTHO,GSL120DGER,GSL120RPAY,GSL121DMOY,GSL121RALL,GSL122DARA,GSL122LBER,GSL122RDOR,GSL123DBER,GSL123RVAL,GSL124DGAR,GSL124RARR,GSL125DLOP,GSL125RRAY,GSL128DCRE,GSL128RCAI,GSL129DMAR,GSL129RPAU,GSL131DALL,GSL131RMON,GSL132DCAM,GSL132RSCH,GSL133DMAA,GSL133LHAR,GSL133RDEA,GSL134DJOH,GSL134LUNS,GSL134RMCC,GSL135DROS,GSL135RMAY,GSL136DBUC,GSL136LCUL,GSL136REVA,GSL137DWU,GSL137LSHA,GSL138DMOR,GSL138RHUL,GSL145DMOR,GSL145RMAB,GSL148DMOR,GSL148LTRO,GSL148RSMI,GSL149DVO,GSL149LBOG,GSL149RTRU,GSL150DBRO,GSL150RSWA,GSSC03DNOW,GSSC03LOXF,GSSC03RLEH,GSSC05DREI,GSSC05RHUD,GSSC09DMAL,GSSC09RYOU,GSU02DGIA,GSU02RHAL,GSU03DRUS,GSU03LLIN,GSU03RNIC,GSU04DBIS,GSU04RCRE,GSU05LEST,GSU05RSCH,GSU08DCOC,GSU08LKLE,GSU08RPAX,GSU09DBUR,GSU09RHAN,GSU12DLY,GSU12RPAR,GSU14DECK,GSU14LHAS,GSU15DWHI,GSU15RVAC,GSU16DJOH,GSU16RCOP,GSU17DBEN,GSU17RHUF,GSU18DTUT,GSU18RKOL,GSU19DGUT,GSU19RGAR,GSU20DHIN,GSU20RWRI,GSU21DZAF,GSU21LDIB,GSU21RDAH,GSU22LSCH,GSU22RBIR,GSU24DJON,

In [72]:
election_columns = election_cols(vtd)
concat['VAP_MOD'] = concat['P0040001-P0050003']
concat['VTD'] = concat['PREC']
concat = concat.reset_index()
df_export = concat[['GEOID20','COUNTYFP20','NAME20','VTD','VAP_MOD']+election_columns+['geometry']]

In [74]:
os.mkdir("./tx_2022_gen_2020_blocks/")
df_export.to_file("./tx_2022_gen_2020_blocks/tx_2022_gen_2020_blocks.shp")

2/18 precincts impacted have population cancelled out my incarcerated pop
Could be explained by VTD data versus precinct

- 439-1603: airport
- 439-1376: Jail/Prison so prion pop>vap
- 133-3111: Appears to be in a park 
- 139-1004: Appears to be roads/neighborhood shape, though no lots drawn in.
- 113-32T0: weird shape, seems plausible noone lives here...
- 085-0202: Contains elementary school + CVS, seems plausible people or no people - just 1 block
- 029-3156

...

Reassign with equal pop assigned by vtd
